In [17]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,classification_report
!pip install six
from six import StringIO
from sklearn.tree import DecisionTreeClassifier 
from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO  
from IPython.display import Image  

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, make_scorer

In [2]:
ad_data = pd.read_csv('train.csv')
ad_data.head()

,UserID,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,UI_9185,Married-civ-spouse,Pharma,Comedy,Male,40,Daytime,United-States,0.027465,Medium,No,0
1,UI_449,Never-married,Political,Comedy,Male,35,Morning,International,0.027465,Low,No,0
2,UI_9014,Never-married,Auto,Comedy,Male,40,Morning,United-States,0.027465,Low,No,0
3,UI_14987,Married-civ-spouse,Pharma,Infomercial,Male,40,Primetime,United-States,0.027465,Low,Yes,0
4,UI_25532,Married-civ-spouse,Other,Comedy,Female,50,Primetime,United-States,0.027465,Low,Yes,1


In [3]:
#Checking data shape

ad_data.shape

(19536, 12)

In [4]:
#Checking data info
ad_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19536 entries, 0 to 19535
Data columns (total 12 columns):
UserID                               19536 non-null object
realtionship_status                  19536 non-null object
industry                             19536 non-null object
genre                                19536 non-null object
targeted_sex                         19536 non-null object
average_runtime(minutes_per_week)    19536 non-null int64
airtime                              19536 non-null object
airlocation                          19536 non-null object
ratings                              19536 non-null float64
expensive                            19536 non-null object
money_back_guarantee                 19536 non-null object
netgain                              19536 non-null int64
dtypes: float64(1), int64(2), object(9)
memory usage: 1.8+ MB


In [5]:
#Desribe data for numerical data
ad_data.describe()

,average_runtime(minutes_per_week),ratings,netgain
count,19536.000000,19536.000000,19536.000000
mean,40.375665,0.038826,0.238022
std,12.483166,0.075971,0.425884
min,1.000000,0.000000,0.000000
25%,40.000000,0.027465,0.000000
50%,40.000000,0.027465,0.000000
75%,45.000000,0.027465,0.000000
max,99.000000,1.000000,1.000000


In [7]:
#Remove user id
ad_data.drop(columns ='UserID', inplace = True)
ad_data.head()

,realtionship_status,industry,genre,targeted_sex,average_runtime(minutes_per_week),airtime,airlocation,ratings,expensive,money_back_guarantee,netgain
0,Married-civ-spouse,Pharma,Comedy,Male,40,Daytime,United-States,0.027465,Medium,No,0
1,Never-married,Political,Comedy,Male,35,Morning,International,0.027465,Low,No,0
2,Never-married,Auto,Comedy,Male,40,Morning,United-States,0.027465,Low,No,0
3,Married-civ-spouse,Pharma,Infomercial,Male,40,Primetime,United-States,0.027465,Low,Yes,0
4,Married-civ-spouse,Other,Comedy,Female,50,Primetime,United-States,0.027465,Low,Yes,1


In [8]:
#Check NaN values

ad_data.isna().sum()

realtionship_status                  0
industry                             0
genre                                0
targeted_sex                         0
average_runtime(minutes_per_week)    0
airtime                              0
airlocation                          0
ratings                              0
expensive                            0
money_back_guarantee                 0
netgain                              0
dtype: int64

In [10]:
#Separate the numerical data and categorical columns
numerical_columns = list(ad_data._get_numeric_data().columns)
numerical_columns



['average_runtime(minutes_per_week)', 'ratings', 'netgain']

In [11]:
#Categorical data
categorical_columns = list(set(ad_data.columns) - set(ad_data._get_numeric_data().columns))
categorical_columns

['airtime',
 'expensive',
 'money_back_guarantee',
 'genre',
 'airlocation',
 'industry',
 'targeted_sex',
 'realtionship_status']

In [19]:
#converting categorical to numerical value by encoding them
for column in ['airtime','expensive','money_back_guarantee','genre','airlocation', 'industry', 'targeted_sex', 'realtionship_status']:
    ad_data = pd.concat((ad_data,pd.get_dummies(ad_data[column])),1)
    ad_data.drop(column,1,inplace = True)
ad_data

,average_runtime(minutes_per_week),ratings,netgain,Daytime,Morning,Primetime,High,Low,Medium,No,...,Political,Female,Male,Divorced,Married-AF-spouse,Married-civ-spouse,Married-spouse-absent,Never-married,Separated,Widowed
0,40,0.027465,0,1,0,0,0,0,1,1,...,0,0,1,0,0,1,0,0,0,0
1,35,0.027465,0,0,1,0,0,1,0,1,...,1,0,1,0,0,0,0,1,0,0
2,40,0.027465,0,0,1,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,0
3,40,0.027465,0,0,0,1,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,50,0.027465,1,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19531,40,0.027465,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
19532,15,0.027465,1,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,1,0,0
19533,55,0.027465,0,0,0,1,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
19534,38,0.027465,0,0,0,1,1,0,0,0,...,1,0,1,0,0,0,0,1,0,0


In [12]:
#Checking Class imbalance
print(ad_data.netgain.value_counts())
def class_imbalance(target):
    class_values = (target.value_counts()/target.value_counts().sum())*100
    return class_values

class_imbalance(ad_data['netgain'])

0    14886
1     4650
Name: netgain, dtype: int64


0    76.197789
1    23.802211
Name: netgain, dtype: float64

In [20]:
#declaring features and target variable
X = ad_data.drop('netgain',1)
y = ad_data.netgain
#splitting data into train and test 
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 0, test_size = 0.4)

In [21]:
#Creating model using DT
DT_clf_gini = DecisionTreeClassifier(criterion='gini')
DT_clf_gini.fit(X_train, y_train)

y_pred_gini = DT_clf_gini.predict(X_test)

#Checking accuracy
accuracy_gini = accuracy_score(y_test, y_pred_gini)

accuracy_gini

0.7965451055662188

In [22]:
#Creating random forest
clf_1 = RandomForestClassifier(min_samples_leaf=2, random_state=9)
clf_1.fit(X_train,y_train)

# predicting for min_samples 0f leaf as 2.
y_clf_1 = clf_1.predict(X_test)

#calculate accuracy and roc auc score for min leaf samples

accuracy_clf_1 = accuracy_score(y_test, y_clf_1)
auc_roc1 = roc_auc_score(y_test, y_clf_1)
print("Accuracy score of the model with min sample leaf 2 is: {}".format(accuracy_clf_1))
print("roc_auc_score of the model1 is: {}".format(auc_roc1))


Accuracy score of the model with min sample leaf 2 is: 0.819961612284069
roc_auc_score of the model1 is: 0.6896989077863637
